# Model training

## Import packages

In [244]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import os

## Read data

In [245]:
df = pd.read_csv('data/out.csv')

## Transform df to get array of poems (1 input to start with)

In [246]:
df = df[['0', '1', '2']].agg(lambda x: '\n'.join(x.values), axis=1)
array_of_poems = df.values.tolist()

### Show array

In [247]:
for i, sentence in enumerate(array_of_poems):
    if i == 3:
        break;
    print(sentence)
    print('\n')

last red in the sky
a small girls moon face rises
over the counter


christmas services
a cellular phone rings out
handels messiah


passover darkness 
before the buds burst open
a childs eyes in death




## Extend poems to match longest poem to get unified length

In [248]:
maxlen = len(max(array_of_poems, key = len))

for i in range(len(array_of_poems)):
    array_of_poems[i] = array_of_poems[i].ljust(maxlen) 

### Show array

In [249]:
for i, sentence in enumerate(array_of_poems):
    if i == 3:
        break;
    print(len(sentence))
    print('\n')

103


103


103




## Get unique chars in corpus

In [250]:
vocab = sorted(set(''.join(array_of_poems)))
print(f'{len(vocab)} unique chars')

29 unique chars


In [251]:
vocab

['\n',
 ' ',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

## Before training, convert strings to numerical representation

In [6]:
chars = tf.strings.unicode_split(array_of_poems, input_encoding='UTF-8')
chars

2022-04-25 10:34:12.736829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 10:34:12.782502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 10:34:12.784533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-25 10:34:12.785382: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

<tf.RaggedTensor [[b'l', b'a', b's', ..., b' ', b' ', b' '],
 [b'c', b'h', b'r', ..., b' ', b' ', b' '],
 [b'p', b'a', b's', ..., b' ', b' ', b' '],
 ...,
 [b'a', b'i', b'n', ..., b' ', b' ', b' '],
 [b'i', b's', b' ', ..., b' ', b' ', b' '],
 [b'w', b'a', b'n', ..., b' ', b' ', b' ']]>

### Create layers for model to transform ids from chars and vice versa

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[19, 8, 26, ..., 2, 2, 2],
 [10, 15, 25, ..., 2, 2, 2],
 [23, 8, 26, ..., 2, 2, 2],
 ...,
 [8, 16, 21, ..., 2, 2, 2],
 [16, 26, 2, ..., 2, 2, 2],
 [30, 8, 21, ..., 2, 2, 2]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars

<tf.RaggedTensor [[b'l', b'a', b's', ..., b' ', b' ', b' '],
 [b'c', b'h', b'r', ..., b' ', b' ', b' '],
 [b'p', b'a', b's', ..., b' ', b' ', b' '],
 ...,
 [b'a', b'i', b'n', ..., b' ', b' ', b' '],
 [b'i', b's', b' ', ..., b' ', b' ', b' '],
 [b'w', b'a', b'n', ..., b' ', b' ', b' ']]>

### Function to join chars back to string

In [9]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Create training example and targets

In [10]:
poems = ''.join(array_of_poems)

In [11]:
all_ids = ids_from_chars(tf.strings.unicode_split(poems, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(93):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

l
a
s
t
 
r
e
d
 
i
n
 
t
h
e
 
s
k
y


a
 
s
m
a
l
l
 
g
i
r
l
s
 
m
o
o
n
 
f
a
c
e
 
r
i
s
e
s


o
v
e
r
 
t
h
e
 
c
o
u
n
t
e
r
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [13]:
seq_length = 92
examples_per_epoch = 10

In [14]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

In [15]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'last red in the sky\na small girls moon face rises\nover the counter                          '
Target: b'ast red in the sky\na small girls moon face rises\nover the counter                           '


## Create training batches

In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 92), dtype=tf.int64, name=None), TensorSpec(shape=(64, 92), dtype=tf.int64, name=None))>

## Build model

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 256

In [19]:
class MyModel(tf.keras.Model):
      def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(rnn_units,
                                         return_sequences=True,
                                         return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    
      def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.lstm.get_initial_state(x)
        x, *states = self.lstm(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [20]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try out

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

2022-04-25 10:34:49.090754: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


(64, 92, 34) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  8704      
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 dense (Dense)               multiple                  34850     
                                                                 
Total params: 5,290,530
Trainable params: 5,290,530
Non-trainable params: 0
_________________________________________________________________


In [23]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 92, 34)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.525304, shape=(), dtype=float32)


In [24]:
tf.exp(example_batch_mean_loss).numpy()

33.964104

In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, use_multiprocessing=True, callbacks=[checkpoint_callback])

Epoch 1/10
1459/1459 [==============================] - 42s 27ms/step - loss: 1.2989
Epoch 2/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.8885
Epoch 3/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.8154
Epoch 4/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.7786
Epoch 5/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.7518
Epoch 6/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.7299
Epoch 7/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.7105
Epoch 8/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.6925
Epoch 9/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.6759
Epoch 10/10
1459/1459 [==============================] - 40s 27ms/step - loss: 0.6604


In [132]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.6):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [133]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [158]:
from typing import List

def generateHaikus(charList: List):
    start = time.time()
    states = None
    next_char = tf.constant(charList)
    result = [next_char]

    for n in range(93):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()
    print(result[0].numpy().decode('utf-8'))

In [165]:
for el in vocab:
    generateHaikus([el])
    print('\n')


just watched the issue make it
spend time with your boo                                      


                                                                                              


0ate less than anyone
who really cant handle the
time he was a bit                            


1_00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


3the best thing about
me is having a pair of
point in the morning                             


9emes if someone comes
to the truth if you do that
field some day today                       


_ri and my ethict is
a game now i have to watch
the best for a word                           


apparently its
fair has to be a shit now
i just want to die                                   


but you gotta thank
you for what you did to be
a day forever                                  


can agree with the
royal wedding there are for
the rest of the day                            


dont worry it is
a lot of conv

In [156]:
start = time.time()
states = None
next_char = tf.constant(['y'])
result = [next_char]

for n in range(93):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'))
print('\nRun time:', end - start)

you do not believe
in yourself but you have to
go to the beach too                            

Run time: 0.21434712409973145
